# Noa Pereira Prada Schnor

# Machine Learning and Statistics Module

# GMIT

This assessment concerns the well-known Boston House Prices dataset and the Python packages scipy, keras, and jupyter. The project consists in five sections: 1. Introduction of the Boston House Price Dataset; 2. Description - descriptive statistics and plots to describe the Boston House Price dataset; 3. Inference/Analysis - using inferential statistics to analyse whether there is a significant difference in median house prices between houses that are along the Charles river and those that aren’t; 4. Prediction - using keras to create a neural network that can predict the median house price based on the other variables in the dataset and 5. Conclusions.

1. Description of the Boston House Price Dataset

The dataset was first published in 1978 contains US census data concerning houses in various areas around the city of Boston. Each sample (row) corresponds to a unique area and has about 13 measures (variables/columns).

2. Descriptive

In [ ]:
#import libraries
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
from scipy.stats import mstats
from scipy.stats import kruskal
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#Load the Boston Housing dataset from sklearn.datasets
df = load_boston()

In [ ]:
# Description of the dataset
df.DESCR

In [ ]:
#Shape - visualization of the length and the breadth of the dataset
df.data.shape

In [ ]:
#Conversion to numpy array - target is the variable MEDV (input is the price and the outputs are the 'feature names')

df_bsn = pd.DataFrame(df.data,columns=df.feature_names)
df_bsn['target'] = pd.Series(df.target)

df_bsn = pd.DataFrame(df['data'], columns=df['feature_names'])
df_bsn['target'] = df['target']

In [ ]:
#Checking missing data
df_bsn.isnull().sum()

In [ ]:
#Check the first rows
df_bsn.head()

In [ ]:
#Return a Numpy representation of the DataFrame - check if the dataset is stored in an array
df_bsn.values

In [ ]:
#Summary and description of the dataset to get a detailed statistical information for each column

df_bsn.describe()

In [ ]:
sns.pairplot(df_bsn, hue="CHAS", markers=["o", "s"])

In [ ]:
sns.pairplot(df_bsn, hue="CHAS", vars=["CRIM", "LSTAT", "B", "DIS", "ZN", "RM", "target"])

Looking at the target (price) it looks like that the variable CRIM (per capita crime rate by town) does not seem to have a strong relationship with price regarding the area traits the River or not as most of the areas that the tract bounds the River have a value nearly 0 of CRIM and the price varies a lot. However, when the areas do not bounds the River it looks like the price tends to be higher when the CRIM value is low.
Most of the areas that bounds the River have a value of B around 400 and it looks like the price varies a lot in areas with the same value of B.
Mainly areas that bounds the River have a value of 0 for ZN (proportion of residential land zoned for lots over 25,000 sq.ft.) and even with the same value of ZN the price varies. 
The LSTAT (% lower status of the population) variable seems to be negatively correlated and the variable RM (average number of rooms per dwelling) positively correlated with price for CHAS equal to 1 or 0.

In [ ]:
#Checking for outliers

sns.boxplot(x="CHAS", y="target", data=df_bsn)
plt.show()


The variable target (price) is skewed to the right. Furthermore, it looks like that there are some unusually high median-values in the data, especially in the areas that do not bounds the River. To confirm it there is a QQ Plot of those variables below.

In [ ]:
pr = sm.qqplot(df_bsn["target"])

dt = pr.findobj(lambda x: hasattr(x, 'get_color') and x.get_color() == 'b')

[d.set_alpha(0.3) for d in dt]

The QQ-Plot shows that upper-third seems to come from a different distribution than the lower two-thirds.  Therefore, the median-values of price aren’t normally distributed.

In [ ]:
#Split the input and the output variables

prices = df_bsn['target']
features = df_bsn.drop('target', axis = 1)

In [ ]:
df_bsn.corr(method='pearson')

In [ ]:
# Heatmap pf Pearson Correlation with no redundant mappings

corr = df_bsn.corr(method='pearson')
fig, ax = plt.subplots(figsize=(9, 9))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

with sns.axes_style("white"):
     ax = sns.heatmap(corr, annot=True, fmt=".2f", mask=mask)
plt.show()

From the Pearson Correlation the attributes LSTAT, RM, and PTRATIO seem to have good correlation with the variable price. These variables maybe can be considered when optimizing the predictive value for price.

In [ ]:
plt.figure(figsize=(15, 5))

attributes = ['LSTAT', 'RM', 'PTRATIO','CHAS']
price = df_bsn['target']

for i, col in enumerate(attributes):
    plt.subplot(1, len(attributes) , i+1)
    x = df_bsn[col]
    y = price
    plt.scatter(x, y, c='g', marker='+')
    plt.xlabel(col)
    plt.ylabel('House prices in $1K')

In [ ]:
#Check the raking of all attributes using linear regression as a model

data = df["data"]
price = df["target"]
attributes = df["feature_names"]
 
linear = LinearRegression()


rfe = RFE(linear, n_features_to_select=1)
rfe.fit(data,price)
sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), attributes))

In [ ]:
#Check number of areas that the land tract bounds Charles River
df_bsn['CHAS'].value_counts()

There are 35 neighborhoods on the Charles river.

In [ ]:
#Setting the variable CHAS as an integer
df_bsn['CHAS'] = df_bsn['CHAS'].astype('int64')

In [ ]:
#Confirming the type and size of each attribute
df_bsn.dtypes

In [ ]:
#Splitting the dataset in 2 groups related to CHAS variable
river = df_bsn[df_bsn['CHAS'] == 1.0]
noriver = df_bsn[df_bsn['CHAS'] == 0]

In [ ]:
river.describe().T

In [ ]:
noriver.describe().T

Do the samples river and noriver have the same distribution? To get the answer it was performed the Mann Whitney U test and the Kruskal Wallis H test using scipy functions. These statistical tests are the the nonparametric version of the (paired) Student t-test.

In [ ]:
#Mann Whitney U test or Wilcoxon-Mann Whitney test
scipy.stats.mannwhitneyu (river,noriver)

The test results show that the samples are likely drawn from samples with differing distributions.

In [ ]:
kruskal(river,noriver)

In [ ]:
river.corr()

When the neighborhood has 1 for the variable CHAS the variables that have a strong correlation with price (target) are RM and LSTAT and a week correlation with ZN, NOX, AGE and B.

In [ ]:
noriver.corr()

When the neighborhood has 0 for the variable CHAS the variables that have a strong correlation with price (target) are RM and LSTAT, a moderate correlation with CRIM, INDUS, NOX, AGE, RAD, TAX and PTRATIO and a week correlation with ZN, DIS and B.

### Neural Network using keras

In [ ]:
#Split the dataset into input features (i) and the feature we wish to predict - price (p)

i = df_bsn.iloc[:,0:13] #assign the first 13 columns of our array to a variable i
p = df_bsn.iloc[:,13]

In [ ]:
#Make sure that the scale of the input features are similar as some of the dataset features have different scale and it makes difficult to for the initialization of the neural network

minmaxscaler = preprocessing.MinMaxScaler()
i_scale = minmaxscaler.fit_transform(i)

In [ ]:
# Split the dataset into a training set, a validation set and a test set (in total 6 variables i_train, i_val, i_test, p_train,p_val and p_test)

i_train, i_val_and_test, p_train, p_val_and_test = train_test_split(i_scale, p, test_size=0.3) #val_and_test size will be 30% of the overall dataset
i_val, i_test, p_val, p_test = train_test_split(i_val_and_test, p_val_and_test, test_size=0.5) #val_ and _test split equally to the validation set and the test set

#Check the shapes of the arrays
i_train.shape, i_val.shape, i_test.shape, p_train.shape, p_val.shape, p_test.shape

 Variable _train counts for 70% of full dataset, _val for 15% and _test for 15%.

In [ ]:
#Setting up the architecture - first and second layer as a dense (fully-connected) layers with 64 neurons, ReLU activation and the input shape is 13 and the last layer layer is a dense layer with 1 neuron

model = Sequential([Dense(64, kernel_initializer='normal', activation='relu', input_shape=(13,)),Dense(64, kernel_initializer='normal',activation='relu'), Dense(1, kernel_initializer='normal'),
])
model.compile(optimizer='rmsprop',loss='mse',metrics=['adam'])

In [ ]:
#Estimate of the model’s performance
kf = KFold(n_splits=10)
rslts = cross_val_score(estimator, X, Y, cv=kf)
rslts.mean()
rslts.std()

#mean squared error including the average and standard deviation (average variance) across all 10 folds of the cross validation evaluation.

In [ ]:
#Training on the data

h = model.fit(i_train, p_train, batch_size=10, epochs=100, validation_data=(i_val, p_val))

In [ ]:
#Find the accuracy of the test set - the aim is to get the test accuracy anywhere between 80% to 95%

model.evaluate(i_test, p_test)[1]